## Mongo

In [ ]:
# brew services start mongodb-community@6.0
# on command line

In [ ]:
import os
import time
import json
import math
import pickle
import requests
import scipy as sp
import numpy as np
import pprint as pp
import pandas as pd
import pymongo as pm
import powerlaw as pl
import datetime as dt
import plotly.colors as pc
import plotly.graph_objects as go
from web3 import Web3, contract

In [ ]:
client = pm.MongoClient()
db = client.ronin
client, db

In [ ]:
db.axiesales

## Lunar Express

**Lunar Express Items Repository**

https://explorer.roninchain.com/address/ronin:144697847f4bf184534af5945abe0fb5f1b14fba

**Lunar Express Axie Repository**

https://explorer.roninchain.com/address/ronin:27263b825bab7ef905718185342905fbd248d1ed

**Example Lunar Express Burn Transaction**

https://explorer.roninchain.com/tx/0x679d06b10f760cb7eb07f25718278838f53f25be5139ad607555db4fa639212a

**Personal LE Item Receipt**

https://explorer.roninchain.com/tx/0x64ee9f6c4bc4cad1c7cbc5b800f6a22bae916f190e2d63c2ee48c1ac6758d23b

In [ ]:
contract_address_LE_items = Web3.toChecksumAddress('0x144697847f4bf184534af5945abe0fb5f1b14fba')
contract_address_LE_axies = Web3.toChecksumAddress('0x27263b825bab7ef905718185342905fbd248d1ed')
contract_address_axie = Web3.toChecksumAddress('0x32950db2a7164ae833121501c797d79e7b79d74c')
contract_address_axs = Web3.toChecksumAddress('0x97a9107c1793bc407d6f527b77e7fff4d812bece')
contract_address_slp = Web3.toChecksumAddress('0xa8754b9fa15fc18bb59458815510e40a12cd2014')
contract_address_LE_items, contract_address_LE_axies,\
    contract_address_axie, contract_address_axs, contract_address_slp

In [ ]:
'request to get LE items', db.transactions.count_documents({'to': contract_address_LE_items.lower()})
# 'from' is users

In [ ]:
LE_item_mints = []
cursor = db.transactions.find({'to': contract_address_LE_items.lower()})
for tx in cursor:
    LE_item_mints.append(tx)
LE_item_mints = pd.DataFrame(LE_item_mints)

In [ ]:
LE_axie_burns = []
cursor = db.axietransfers.find({'to': contract_address_LE_axies.lower()})
for tx in cursor:
    LE_axie_burns.append(tx)
LE_axie_burns = pd.DataFrame(LE_axie_burns)
LE_axie_burns['date'] = LE_axie_burns.apply(
    lambda row : row['created_at'].date(), axis = 1)
LE_axie_burns = LE_axie_burns[LE_axie_burns['date'] <= dt.date(2022, 2, 20)]
LE_axie_burns[:4]

In [ ]:
usage_LE = LE_axie_burns.groupby(by=['date', 'from']).count()\
    .reset_index()\
    .filter(['date', 'to'])\
    .rename(columns={'to': 'count'})
usage_by_date_LE = usage_LE.groupby(by=['date']).count()\
    .reset_index()
usage_by_date_LE[:4]

In [ ]:
cursor = db.summeraxies.find()
summer_axies = pd.DataFrame()
for tx in cursor:
    if tx['breedTx']==None:
        continue
    axie_attributes = {
        'axieId': tx['id'],
        'numSummer': tx['numSummer'],
        'numShiny': tx['numShiny'],
        'createdAt': tx['breedTx']['createdAt'],
        'createDate': tx['breedTx']['createdAt'].date(),
        'txHash': tx['breedTx']['hash'],
        'breeder': tx['breedTx']['from'],
        'sireId': tx['breedTx']['sireId'],
        'matronId': tx['breedTx']['matronId'],
        'feeAxs': tx['breedTx']['feeAxs'],
        'feeSlp': tx['breedTx']['feeSlp']
    }
    for part in tx['parts']:
        axie_attributes[part['type']] = part['id']
    summer_axies = pd.concat(
        [summer_axies, pd.DataFrame(axie_attributes, index=[0])],
        ignore_index=True
    )
summer_axies[:4]

In [ ]:
LE_axie_burns[:4]

In [ ]:
breeders = [x.lower() for x in summer_axies['breeder'].values]
burners_in_breeders = [(x in breeders) for x in LE_axie_burns['from'].values]

In [ ]:
LE_axie_burns[burners_in_breeders]

In [ ]:
usage_LE_breeders_only = LE_axie_burns[burners_in_breeders].groupby(by=['date', 'from']).count()\
    .reset_index()\
    .filter(['date', 'to'])\
    .rename(columns={'to': 'count'})
usage_by_date_LE_breeders_only = usage_LE_breeders_only.groupby(by=['date']).count()\
    .reset_index()
usage_by_date_LE_breeders_only[:4]

In [ ]:
usage_by_date_LS = summer_axies.groupby(by='createDate').count()\
    .reset_index()\
    .filter(['createDate', 'axieId'])\
    .rename(columns={'axieId': 'count'})
usage_by_date_LS[:4]

[summer axie pic](https://bucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com/public/images/7dd59e22-1b86-4302-9368-8523b1524afb_2860x1580.png)

In [ ]:
usage_by_date_est_LS = usage_by_date_LS.copy(deep=True)
for i, axie in summer_axies.iterrows():
    if axie['Horn']=='horn-vanilla-ice-cream':
        index = usage_by_date_est_LS['createDate']==axie['createDate']
        usage_by_date_est_LS.loc[index, 'count'] += 8
    if axie['Back']=='back-turtle-buoy':
        index = usage_by_date_est_LS['createDate']==axie['createDate']
        usage_by_date_est_LS.loc[index, 'count'] += 8
# This doesn't work because the later axies inherit vanilla icecream and turtle buoy.

In [ ]:
first_trough = dt.date(2022, 7, 18) # trough of first breeding cycle
first_cycle = summer_axies[summer_axies['createDate'] < first_trough]
count_vanilla_icecream = len(
    first_cycle[first_cycle['Horn']=='horn-vanilla-ice-cream'].index)
count_turtle_buoy = len(first_cycle[first_cycle['Back']=='back-turtle-buoy'].index)
count_first_cycle = len(first_cycle.index)
print("vanilla icecream: " +\
      f"{count_vanilla_icecream}/{count_first_cycle} = " +\
      f"{count_vanilla_icecream/count_first_cycle:.2}")
print("turtle buoy: " +\
      f"{count_turtle_buoy}/{count_first_cycle} = " +\
      f"{count_turtle_buoy/count_first_cycle:.2}")

In [ ]:
def monoExp(x, m, t, b):
    return m * np.exp(-t * x) + b

days_LE = np.array([float(x.days) for x in usage_by_date_LE['date']-usage_by_date_LE['date'][0]])
count_LE = usage_by_date_LE['count']
p0 = (20000, 1, 1000)
params_LE, cv = sp.optimize.curve_fit(monoExp, days_LE, count_LE, p0)
m, t, b = params_LE
print(f"m={m:.2f}, t={t:.2f}, b={b:.2f}")
squaredDiffs = np.square(count_LE - monoExp(days_LE, m, t, b))
squaredDiffsFromMean = np.square(count_LE - np.mean(count_LE))
rSquared = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)
print(f"R² = {rSquared}")

In [ ]:
def monoExp(x, m, t, b):
    return m * np.exp(-t * x) + b

days_LS = np.array([float(x.days) for x in usage_by_date_LS['createDate']-usage_by_date_LS['createDate'][0]])
count_LS = usage_by_date_LS['count']
p0 = (1000, .1, 50)
params_LS, cv = sp.optimize.curve_fit(monoExp, days_LS, count_LS, p0)
m, t, b = params_LS
print(f"m={m:.2f}, t={t:.2f}, b={b:.2f}")
squaredDiffs = np.square(count_LS - monoExp(days_LS, m, t, b))
squaredDiffsFromMean = np.square(count_LS - np.mean(count_LS))
rSquared = 1 - np.sum(squaredDiffs) / np.sum(squaredDiffsFromMean)
print(f"R² = {rSquared}")

In [ ]:
usage_LE['count'].sum()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=usage_by_date_LE['date'],
        y=usage_by_date_LE['count'],
        line_color=pc.qualitative.Plotly[0],
        showlegend=False
    )
)
# fig.add_trace(
#     go.Scatter(
#         x=usage_by_date_LE['date'],
#         y=monoExp(days_LE, params_LE[0], params_LE[1], params_LE[2]),
#         line={
#             'color': pc.qualitative.Plotly[0],
#             'dash': 'dash'
#         },
#         showlegend=False
#     )
# )
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis_title='usage',
    title='Usage (Lunacian Express)'
)
fig.show()
fig.write_image("images/craft_usage_time_le.png", scale=3)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=usage_by_date_LS['createDate'],
        y=usage_by_date_LS['count'],
        line={'color': pc.qualitative.Plotly[1],},
        showlegend=False
    )
)
# fig.add_trace(
#     go.Scatter(
#         x=usage_by_date_LS['createDate'],
#         y=monoExp(days_LS, params_LS[0], params_LS[1], params_LS[2]),
#         line={'color': pc.qualitative.Plotly[1],},
#         showlegend=False
#     )
# )
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis_title='usage', title='Breeds (Lunacian Summer)'
)
fig.show()
fig.write_image("images/craft_usage_time_ls.png", scale=3)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=usage_by_date_LE_breeders_only['date'],
        y=usage_by_date_LE_breeders_only['count'],
        line_color=pc.qualitative.Plotly[0],
        showlegend=False
    )
)
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis_title='usage',
    title='Usage (Lunacian Express, breeders only)'
)
fig.show()
fig.write_image("images/craft_usage_time_le_breeders_only.png", scale=3)

In [ ]:
db.axietransfers.find_one({
    'from': '0x0000000000000000000000000000000000000000',
    'created_at': {'$gt': dt.datetime(2022, 1, 20)}
})

In [ ]:
db.axietransfers.find_one({
    'from': '0x0000000000000000000000000000000000000000',
    'created_at': {'$gt': dt.datetime(2022, 7, 14)}
})

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LE['date'] - usage_by_date_LE['date'][0]],
        y=usage_by_date_LE['count'],
        line_color=pc.qualitative.Plotly[0],
        name='Lunacian Express'
    )
)
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LS['createDate'] - usage_by_date_LS['createDate'][0]],
        y=usage_by_date_LS['count'],
        line_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer'
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis_title='days',
    yaxis_title='usage',
    yaxis2={'side': 'right'}
)
fig.show()
# fig.write_image("images/usage_time_absolute.png", scale=3)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LE['date'] - usage_by_date_LE['date'][0]],
        y=usage_by_date_LE['count'],
        line_color=pc.qualitative.Plotly[0],
        name='Lunacian Express',
    )
)
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LS['createDate'] - usage_by_date_LS['createDate'][0]],
        y=usage_by_date_LS['count'],
        line_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer',
        yaxis='y2'
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis_title='days',
    yaxis={'title': 'usage', 'side': 'left'},
    yaxis2={'side': 'right', 'overlaying': 'y'}
)
fig.show()
# fig.write_image("images/usage_time_scaled.png", scale=3)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LE['date'] - usage_by_date_LE['date'][0]],
        y=usage_by_date_LE['count'],
        line_color=pc.qualitative.Plotly[0],
        name='Lunacian Express',
    )
)
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LS['createDate'] - usage_by_date_LS['createDate'][0]],
        y=usage_by_date_LS['count'],
        line_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer',
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis_title='days',
    yaxis={'title': 'usage', 'side': 'left', 'type': 'log'}
)
fig.show()
# fig.write_image("images/usage_time_log.png", scale=3)

In [ ]:
breeder_count = summer_axies.groupby(by='breeder').count()\
    .reset_index()\
    .filter(['breeder', 'numSummer'])\
    .rename(columns={'numSummer': 'count'})
pdf_LS = pl.pdf(breeder_count['count'])
fit_LS = pl.Fit(breeder_count['count'], discrete=True)
# fit_LS_x = np.logspace(np.log10(1), np.log10(breeder_count['count'].max()), 30)
# fit_LS_y = fit_LS_x ** (-1 * fit_LS.power_law.alpha)
fit_LS_x = np.logspace(
    np.log10(fit_LS.xmin),
    np.log10(breeder_count['count'].max()-fit_LS.xmin),
    30
)
fit_LS_y = (fit_LS.alpha-1) / (fit_LS.xmin) * \
    (fit_LS_x/fit_LS.xmin) ** (-fit_LS.alpha)
fit_LS_y = fit_LS_y * pdf_LS[1][2] / fit_LS_y[0]


print(f"\nɑ={fit_LS.power_law.alpha:.2}, xmin={fit_LS.power_law.xmin:.1f}")

breeder_count[:4]

In [ ]:
# usage_user_LE = LE_axie_burns.groupby(by=['from']).sum()\
#     .reset_index()\
#     .sort_values(by='count', ascending=False)
usage_user_LE = LE_axie_burns.filter(['from', '_id'])\
    .groupby(by=['from']).count()\
    .reset_index()\
    .rename(columns={'_id': 'count'})
pdf_LE = pl.pdf(usage_user_LE['count'])
fit_LE = pl.Fit(usage_user_LE['count'], discrete=True)
# fit_LE_x = np.logspace(np.log10(1), np.log10(usage_user_LE['count'].max()), 30)
# fit_LE_y = fit_LE_x ** (-1 * fit_LE.power_law.alpha)
fit_LE_x = np.logspace(
    np.log10(fit_LE.xmin),
    np.log10(usage_user_LE['count'].max()-fit_LE.xmin),
    30
)
fit_LE_y = (fit_LE.alpha-1) / (fit_LE.xmin) * \
    (fit_LE_x/fit_LE.xmin) ** (-fit_LE.alpha)
fit_LE_y = fit_LE_y * pdf_LE[1][4] / fit_LE_y[0]


print(f"\nɑ={fit_LE.power_law.alpha:.2}, xmin={fit_LE.power_law.xmin:.1f}")

usage_user_LE[:4]

In [ ]:
usage_user_LE_breeders_only = LE_axie_burns[burners_in_breeders].filter(['from', '_id'])\
    .groupby(by=['from']).count()\
    .reset_index()\
    .rename(columns={'_id': 'count'})
pdf_LE_breeders_only = pl.pdf(usage_user_LE_breeders_only['count'])
fit_LE_breeders_only = pl.Fit(usage_user_LE_breeders_only['count'], discrete=True)
fit_LE_breeders_only_x = np.logspace(
    np.log10(fit_LE_breeders_only.xmin),
    np.log10(usage_user_LE_breeders_only['count'].max()-fit_LE_breeders_only.xmin),
    30
)
fit_LE_breeders_only_y = (fit_LE_breeders_only.alpha-1) / (fit_LE_breeders_only.xmin) * \
    (fit_LE_breeders_only_x/fit_LE_breeders_only.xmin) ** (-fit_LE_breeders_only.alpha)
fit_LE_breeders_only_y = fit_LE_breeders_only_y * pdf_LE_breeders_only[1][9] / fit_LE_breeders_only_y[0]


print(f"\nɑ={fit_LE_breeders_only.power_law.alpha:.2}, xmin={fit_LE_breeders_only.power_law.xmin:.1f}")

usage_user_LE_breeders_only[:4]

In [ ]:
sp.stats.ks_2samp(usage_user_LE['count'], breeder_count['count'])

In [ ]:
sp.stats.ks_2samp(usage_user_LE_breeders_only['count'], breeder_count['count'])

In [ ]:
sp.stats.ks_2samp(usage_user_LE['count'], usage_user_LE_breeders_only['count'])

In [ ]:
fit_a2_x = np.logspace(np.log10(1), np.log10(usage_user_LE['count'].max()), 30)
fit_a2_y = fit_LE_x ** (-2)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=pdf_LE[0], y=pdf_LE[1],
        mode='markers', marker_color=pc.qualitative.Plotly[0],
        name='Lunacian Express'
    )
)
fig.add_trace(
    go.Scatter(
        x=fit_LE_x,
        y=fit_LE_y,
        mode='lines', marker_color=pc.qualitative.Plotly[0],
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x=pdf_LE_breeders_only[0], y=pdf_LE_breeders_only[1],
        mode='markers', marker_color=pc.qualitative.Plotly[2],
        name='Lunacian Express (breeders only)'
    )
)
fig.add_trace(
    go.Scatter(
        x=fit_LE_breeders_only_x,
        y=fit_LE_breeders_only_y,
        mode='lines', marker_color=pc.qualitative.Plotly[2],
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x=pdf_LS[0], y=pdf_LS[1],
        mode='markers', marker_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer'
    )
)
fig.add_trace(
    go.Scatter(
        x=fit_LS_x,
        y=fit_LS_y,
        mode='lines', marker_color=pc.qualitative.Plotly[1],
        showlegend=False
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':40, 'b':10},
    yaxis={'title': 'P(uses)', 'type': 'log'},
    xaxis={'title': 'usage per user', 'type': 'log'},
    title='Crafting usage'
)
fig.show()
fig.write_image("images/craft_usage_pdf.png", scale=3)

In [ ]:
pdf_daily_LE = pl.pdf(usage_LE['count'])
fit_daily_LE = pl.Fit(usage_LE['count'], discrete=True)
# fit_daily_LE_x = np.logspace(np.log10(1), np.log10(usage_LE['count'].max()), 30)
# fit_daily_LE_y = fit_daily_LE_x ** (-1 * fit_daily_LE.alpha)
fit_daily_LE_x = np.logspace(
    np.log10(fit_daily_LE.xmin),
    np.log10(usage_LE['count'].max()-fit_daily_LE.xmin),
    30
)
fit_daily_LE_y = (fit_daily_LE.alpha-1) / (fit_daily_LE.xmin) * \
    (fit_daily_LE_x/fit_daily_LE.xmin) ** (-fit_daily_LE.alpha)
fit_daily_LE_y = fit_daily_LE_y * pdf_daily_LE[1][3] / fit_daily_LE_y[0]


print(f"\nɑ={fit_daily_LE.power_law.alpha:.2}, xmin={fit_daily_LE.power_law.xmin:.1f}")

In [ ]:
pdf_daily_LE_breeders_only = pl.pdf(usage_LE_breeders_only['count'])
fit_daily_LE_breeders_only = pl.Fit(usage_LE_breeders_only['count'], discrete=True)
fit_daily_LE_breeders_only_x = np.logspace(
    np.log10(fit_daily_LE_breeders_only.xmin),
    np.log10(usage_LE_breeders_only['count'].max()-fit_daily_LE_breeders_only.xmin),
    30
)
fit_daily_LE_breeders_only_y = (fit_daily_LE_breeders_only.alpha-1) / (fit_daily_LE_breeders_only.xmin) * \
    (fit_daily_LE_breeders_only_x/fit_daily_LE_breeders_only.xmin) ** (-fit_daily_LE_breeders_only.alpha)
fit_daily_LE_breeders_only_y = fit_daily_LE_breeders_only_y * \
    pdf_daily_LE_breeders_only[1][6] / fit_daily_LE_breeders_only_y[0]
print(f"\nɑ={fit_daily_LE_breeders_only.power_law.alpha:.2}, " +\
      f"xmin={fit_daily_LE_breeders_only.power_law.xmin:.1f}")

In [ ]:
daily_LS = summer_axies.groupby(by=['breeder', 'createDate']).count()\
    .reset_index()\
    .filter(['breeder', 'createDate', 'axieId'])\
    .rename(columns={'axieId': 'count'})
daily_LS[:4]

In [ ]:
pdf_daily_LS = pl.pdf(daily_LS['count'])
fit_daily_LS = pl.Fit(daily_LS['count'], discrete=True)
fit_daily_LS_x = np.logspace(
    np.log10(fit_daily_LS.xmin),
    np.log10(daily_LS['count'].max()-fit_daily_LS.xmin),
    30
)
fit_daily_LS_y = (fit_daily_LS.alpha-1) / (fit_daily_LS.xmin) * \
    (fit_daily_LS_x/fit_daily_LS.xmin) ** (-fit_daily_LS.alpha)
fit_daily_LS_y = fit_daily_LS_y * pdf_daily_LS[1][8] / fit_daily_LS_y[0]
print(f"\nɑ={fit_daily_LS.power_law.alpha:.2}, xmin={fit_daily_LS.power_law.xmin:.1f}")

In [ ]:
sp.stats.ks_2samp(usage_LE['count'], daily_LS['count'])

In [ ]:
sp.stats.ks_2samp(usage_LE_breeders_only['count'], daily_LS['count'])

In [ ]:
sp.stats.ks_2samp(usage_LE['count'], usage_LE_breeders_only['count'])

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=pdf_daily_LE[0], y=pdf_daily_LE[1], mode='markers',
        marker_color=pc.qualitative.Plotly[0],
        name='Lunacian Express'
    )
)
fig.add_trace(
    go.Scatter(
        x=fit_daily_LE_x,
        y=fit_daily_LE_y,
        mode='lines',
        marker_color=pc.qualitative.Plotly[0],
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x=pdf_daily_LE_breeders_only[0],
        y=pdf_daily_LE_breeders_only[1],
        mode='markers',
        marker_color=pc.qualitative.Plotly[2],
        name='Lunacian Express (breeders only)'
    )
)
fig.add_trace(
    go.Scatter(
        x=fit_daily_LE_breeders_only_x,
        y=fit_daily_LE_breeders_only_y,
        mode='lines',
        marker_color=pc.qualitative.Plotly[2],
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x=pdf_daily_LS[0],
        y=pdf_daily_LS[1], mode='markers',
        marker_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer'
    )
)
fig.add_trace(
    go.Scatter(
        x=fit_daily_LS_x,
        y=fit_daily_LS_y,
        mode='lines',
        marker_color=pc.qualitative.Plotly[1],
        showlegend=False
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':40, 'b':10},
    yaxis={'title': 'P(daily usage)', 'type': 'log'},
    xaxis={'title': 'daily usage', 'type': 'log'},
    title='Daily usage'
)
fig.show()
fig.write_image("images/craft_usage_daily_pdf.png", scale=3)

In [ ]:
axie_prices = []
wei_per_eth = 1000000000000000000
window_size = 1000
for i in range(32):
    date_start = dt.datetime(2022, 1, 20) + dt.timedelta(days=i)
    date_end = dt.datetime(2022, 1, 20) + dt.timedelta(days=i+1)
    cursor = db.axiesales.find({
        'created_at': {'$gt': date_start, '$lt': date_end},
        'price': {'$ne': '0'}
    })\
        .sort('price', pm.ASCENDING)
    price_eth = 0
    for i in range(window_size):
        row = next(cursor)
        price_eth += int(row['price']) / wei_per_eth
    price_eth /= window_size
    axie_prices.append(price_eth)
axie_prices[:2]

In [ ]:
eth_prices = []
with open('eth_prices.bin', 'rb') as f:
    eth_prices += pickle.load(f)
eth_prices[:2]

In [ ]:
# import requests
# import time

# eth_prices = []
# for i in range(32):
#     date = dt.datetime(2022, 1, 20) + dt.timedelta(days=i)
#     url = "https://api.coingecko.com/api/v3/coins/ethereum/history?date=" + date.strftime("%d-%m-%Y")
#     r = requests.get(url)
#     eth_usd = r.json()['market_data']['current_price']['usd']
#     eth_prices += [eth_usd]
#     time.sleep(2)

In [ ]:
usage_by_date_LE['eth_burned'] = usage_by_date_LE['count'] * axie_prices
usage_by_date_LE['usd_burned'] = usage_by_date_LE['eth_burned'] * eth_prices
usage_by_date_LE[:4]

In [ ]:
total_eth_sunk = usage_by_date_LE['eth_burned'].sum()
total_usd_sunk = usage_by_date_LE['usd_burned'].sum()
print(f"Ξ{total_eth_sunk:,.2f}")
print(f"${total_usd_sunk:,.2f}")

In [ ]:
filename_prices = 'prices_summer_axs_slp.csv'
prices = pd.DataFrame()
if filename_prices in os.listdir('.'):
    with open('prices_summer_axs_slp.csv') as f:
        prices = pd.read_csv(f).drop(['Unnamed: 0'], axis=1)
else:
    summer_dates = summer_axies['createDate'].sort_values().unique()
    axs_prices = []
    slp_prices = []
    for date in summer_dates:
        if str(date) in prices.date.values:
            continue
        url_axs = "https://api.coingecko.com/api/v3/coins/axie-infinity/history?date=" + date.strftime("%d-%m-%Y")
        r = requests.get(url_axs)
        axs_usd = r.json()['market_data']['current_price']['usd']
        axs_prices += [axs_usd]
        url_slp = "https://api.coingecko.com/api/v3/coins/smooth-love-potion/history?date=" + date.strftime("%d-%m-%Y")
        r = requests.get(url_slp)
        slp_usd = r.json()['market_data']['current_price']['usd']
        slp_prices += [slp_usd]
        prices = pd.concat(
            [prices, pd.DataFrame({'date': [str(date)], 'AXS': [axs_usd], 'SLP': [slp_usd]})],
            ignore_index=True
        )
        time.sleep(5)
    # prices.to_csv('prices_summer_axs_slp.csv')
prices[:4]

In [ ]:
fees_daily = summer_axies\
    .filter(['createDate', 'feeAxs', 'feeSlp'])\
    .groupby('createDate').sum()\
    .reset_index()
fees_daily['feeAxsInUsd'] = fees_daily['feeAxs'] * prices['AXS']
fees_daily['feeSlpInUsd'] = fees_daily['feeSlp'] * prices['SLP']
fees_daily = fees_daily[fees_daily['createDate'] > dt.date(2022, 7, 10)]
fees_daily[:4]

In [ ]:
fees_daily[['feeAxsInUsd', 'feeSlpInUsd']].sum()

In [ ]:
fees_daily[['feeAxsInUsd', 'feeSlpInUsd']].sum().sum()

In [ ]:
fees_daily['createDate'][1]

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LE['date'] - usage_by_date_LE['date'][0]],
        y=usage_by_date_LE['usd_burned'],
        line_color=pc.qualitative.Plotly[0],
        name='Lunacian Express',
    )
)
fig.add_trace(
    go.Scatter(
        x=[x.days for x in fees_daily['createDate'] - fees_daily['createDate'][1]],
        y=fees_daily['feeAxsInUsd'] + fees_daily['feeSlpInUsd'],
        line_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer',
        yaxis='y2'
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis_title='days',
    yaxis={'title': 'USD', 'side': 'left'},
    yaxis2={'title':'USD', 'side': 'right', 'overlaying': 'y'},
    title='Value burned'
)
fig.show()
fig.write_image("images/value_burned_y2.png", scale=3)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[x.days for x in usage_by_date_LE['date'] - usage_by_date_LE['date'][0]],
        y=usage_by_date_LE['usd_burned'],
        line_color=pc.qualitative.Plotly[0],
        name='Lunacian Express',
    )
)
fig.add_trace(
    go.Scatter(
        x=[x.days for x in fees_daily['createDate'] - fees_daily['createDate'][1]],
        y=fees_daily['feeAxsInUsd'] + fees_daily['feeSlpInUsd'],
        line_color=pc.qualitative.Plotly[1],
        name='Lunacian Summer',
    )
)
fig.update_layout(
    template='plotly_white', width=540, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    xaxis_title='days',
    yaxis={'title': 'USD', 'side': 'left', 'type': 'log'},
    title='Value burned'
)
fig.show()
fig.write_image("images/craft_value_burned.png", scale=3)

In [ ]:
slp_usage = summer_axies\
    .filter(['createDate', 'feeSlp'])\
    .groupby('createDate').mean().reset_index()
slp_usage['count'] = summer_axies\
    .filter(['createDate', 'feeSlp'])\
    .groupby('createDate').count().reset_index()['feeSlp']
slp_usage['std'] = summer_axies\
    .filter(['createDate', 'feeSlp'])\
    .groupby('createDate').std().reset_index()['feeSlp']
slp_usage['stderr'] = slp_usage['std'] / np.sqrt(slp_usage['count'].values)

fig = go.Figure()
fig.add_trace(go.Scatter(x=slp_usage['createDate'], y=slp_usage['feeSlp']))
# fig.add_trace(go.Scatter(
#     x=slp_usage['createDate'].append(slp_usage['createDate'].sort_values(ascending=False)),
#     y=(slp_usage['feeSlp'] + slp_usage['stderr']).append(slp_usage['feeSlp'] - slp_usage['stderr']),
#     fill='tozerox'
# ))
fig.update_layout(
    template='plotly_white', width=500, height=400,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={
        'title': 'SLP',
        'range': [0, 10000]
#         'range': [0, slp_usage['feeSlp'].max()]
    },
    title='Average SLP used per breed (Lunacian Summer)'
)
fig.show()
fig.write_image("images/craft_avg_slp_per_breed.png", scale=3)